In [13]:
import pandas as pd
import json
from pandas.io.json import json_normalize

import sys
sys.path.insert(1, "./code/")
from Utils import Utils # student's library


In [14]:
train_df = pd.read_csv("./data/train.csv")

In [15]:
# sanity checks
#train_df.id.nunique()
# 19579

#train_df.author.unique()
# array(['EAP', 'HPL', 'MWS'], dtype=object)

In [16]:
Utils.check_for_nulls(train_df)

{'author': 0, 'id': 0, 'text': 0}

In [78]:
short_df = train_df[:20]
short_df.head(1)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP


In [25]:
def tokenize(sentence):
    return sentence.split(' ')

def build_lexicon(sents):
    lex = {}
    for sent in sents:
        words = tokenize(sent)
        for word in words:
            word = word.lower()
            if word in lex:
                lex[word] = lex[word] + 1
            else:
                lex[word] = 1
    return sorted(lex.items(), key=lambda x: x[1], reverse=True)

train_lex = build_lexicon(short_df.text)   
train_lex

[('of', 35),
 ('the', 35),
 ('and', 22),
 ('i', 18),
 ('to', 15),
 ('a', 12),
 ('his', 11),
 ('my', 9),
 ('that', 9),
 ('in', 9),
 ('he', 7),
 ('as', 5),
 ('with', 4),
 ('not', 4),
 ('for', 4),
 ('this', 3),
 ('being', 3),
 ('was', 3),
 ('all', 3),
 ('at', 3),
 ('you', 3),
 ('could', 3),
 ('no', 2),
 ('so', 2),
 ('from', 2),
 ('an', 2),
 ('how', 2),
 ('we', 2),
 ('on', 2),
 ('heart', 2),
 ('but', 2),
 ('very', 2),
 ('had', 2),
 ('almost', 2),
 ('her', 2),
 ('me', 1),
 ('might', 1),
 ('its', 1),
 ('without', 1),
 ('it', 1),
 ('never', 1),
 ('be', 1),
 ('snuff', 1),
 ('which,', 1),
 ('hill,', 1),
 ('took', 1),
 ('air', 1),
 ('looked', 1),
 ('by', 1),
 ('even', 1),
 ('your', 1),
 ('when', 1),
 ('paid', 1),
 ('him', 1),
 ('felt', 1),
 ('here', 1),
 ('say', 1),
 ('long', 1),
 ('little', 1),
 ('eyes', 1),
 ('nor', 1),
 ('shall', 1),
 ('few', 1),
 ('and,', 1),
 ('needed', 1),
 ('native', 1),
 ('they', 1),
 ('their', 1),
 ('well', 1),
 ('process,', 0),
 ('however,', 0),
 ('afforded', 0),
 ('me

In [76]:
# short_sents = [tokenize(x) for x in list(short_df.text)]
# short_sents[0]

In [62]:

import numpy as np
import gensim

# http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/

# this glove code is wrong type for later work
# with open("data/glove.42B.300d.txt", "rb") as lines:
#     w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
#            for line in lines}


# let X be a list of tokenized texts (i.e. list of lists of tokens)
# sentences param is token lists
model = gensim.models.Word2Vec(short_sents, size=100)
w2v = dict(zip(model.wv.index2word, model.wv.vectors))
type(w2v)

dict

In [63]:
w2v.keys()

dict_keys(['of', 'the', 'and', 'I', 'to', 'a', 'his', 'my', 'that', 'in', 'he', 'as', 'with', 'not', 'for'])

In [58]:
#list(w2v.keys())[9990:9999]
# [b'iconic',
#  b'erp',
#  b'crest',
#  b'radius',
#  b'spiral',
#  b'nyse',
#  b'lotion',
#  b'oriental',
#  b'admire']

In [43]:
a_key = list(w2v.keys())[9999]
a_key.decode("utf-8")

'owl'

In [47]:
type(w2v[str.encode('owl')])

numpy.ndarray

In [65]:
# class MeanEmbeddingVectorizer(object):
#     def __init__(self, word2vec):
#         self.word2vec = word2vec
#         # if a text is empty we should return a vector of zeros
#         # with the same dimensionality as all the other vectors
#         self.dim = len(word2vec.itervalues().next())

#     def fit(self, X, y):
#         return self

#     def transform(self, X):
#         return np.array([
#             np.mean([self.word2vec[w] for w in words if w in self.word2vec]
#                     or [np.zeros(self.dim)], axis=0)
#             for words in X
#         ])

In [66]:
# m_e_v = MeanEmbeddingVectorizer(w2v)
# short_X_train = m_e_v.transform(short_sents)

In [67]:
#https://radimrehurek.com/gensim/models/doc2vec.html
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#TaggedDocument does not filter or stem

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(short_sents)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)


In [68]:
vector = model.infer_vector(["system", "response"])
vector

array([-0.08544256,  0.0625724 ,  0.09173509,  0.04404997, -0.05501466],
      dtype=float32)

In [69]:
# this is it! just use the output of infer_vector in training model

numpy.ndarray

In [75]:
# example_tagged_doc = TaggedDocument(short_sents[0], [0])
# example_tagged_doc

In [84]:
# order of ops:
# just grammar features
# just tfidf vectorizer
# both

# kmeans

from sklearn.feature_extraction.text import TfidfVectorizer

corpus = short_df.text
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(corpus)
Y_train = short_df.author
X_test = vectorizer.transform(short_df.text)
Y_test = short_df.author

In [85]:
# linear svc
from sklearn.svm import LinearSVC




lin_clf = LinearSVC()
lin_clf.fit(X_train, Y_train) 


preds = lin_clf.predict(X_test)
preds

array(['EAP', 'HPL', 'EAP', 'MWS', 'HPL', 'MWS', 'EAP', 'EAP', 'EAP',
       'MWS', 'MWS', 'EAP', 'HPL', 'HPL', 'EAP', 'MWS', 'EAP', 'MWS',
       'EAP', 'HPL'], dtype=object)